In [2]:
import findspark
findspark.init("/opt/homebrew/opt/apache-spark/libexec/")


In [4]:
df = spark.read.option("multiLine","true").option("encoding","SJIS").csv("/Users/isomasaki/pyspark_batch/dataset/jinko.csv",header=True,sep=",",inferSchema=False)
df.count()

982

In [5]:
df.show(truncate=False)


+--------------+----------+----+----------+----------+----+------------+----------+----------+
|都道府県コード|都道府県名|元号|和暦（年）|西暦（年）|注  |人口（総数）|人口（男）|人口（女）|
+--------------+----------+----+----------+----------+----+------------+----------+----------+
|00            |全国      |大正|9         |1920      |null|55963053    |28044185  |27918868  |
|01            |北海道    |大正|9         |1920      |null|2359183     |1244322   |1114861   |
|02            |青森県    |大正|9         |1920      |null|756454      |381293    |375161    |
|03            |岩手県    |大正|9         |1920      |null|845540      |421069    |424471    |
|04            |宮城県    |大正|9         |1920      |null|961768      |485309    |476459    |
|05            |秋田県    |大正|9         |1920      |null|898537      |453682    |444855    |
|06            |山形県    |大正|9         |1920      |null|968925      |478328    |490597    |
|07            |福島県    |大正|9         |1920      |null|1362750     |673525    |689225    |
|08            |茨城県    |大正|9       

In [8]:
#日本語のカラム名をローマ字へ変換
from pyspark.sql.types import StructType,StructField,StringType
from pyspark.sql.functions import col

struct = StructType([
    StructField("code",StringType(),False),
    StructField("kenmei",StringType(),False),
    StructField("gengo",StringType(),False),
    StructField("wareki",StringType(),False),
    StructField("seireki",StringType(),False),
    StructField("chu",StringType(),False),
    StructField("sokei",StringType(),False),
    StructField("jinko_male",StringType(),False),
    StructField("jinko_female",StringType(),False)
])

df = spark.read.option("multiLine","true").option("encoding","SJIS") \
.csv("/Users/isomasaki/pyspark_batch/dataset/jinko.csv",header=False,sep=",",inferSchema=False,schema=struct)
df.show()

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|       sokei|jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|
|            00|      全国| 大正|         9|      1920|null|    55963053|  28044185|    27918868|
|            01|    北海道| 大正|         9|      1920|null|     2359183|   1244322|     1114861|
|            02|    青森県| 大正|         9|      1920|null|      756454|    381293|      375161|
|            03|    岩手県| 大正|         9|      1920|null|      845540|    421069|      424471|
|            04|    宮城県| 大正|         9|      1920|null|      961768|    485309|      476459|
|            05|    秋田県| 大正|         9|      1920|null|      898537|    453682|      444855|
|            06|    山形県| 大正|         9|      1920|null|      968925|    478328|      49059

In [9]:
df.where(df.gengo == "平成").show()


+----+----------------------+-----+------+-------+----+---------+----------+------------+
|code|                kenmei|gengo|wareki|seireki| chu|    sokei|jinko_male|jinko_female|
+----+----------------------+-----+------+-------+----+---------+----------+------------+
|  00|                  全国| 平成|     2|   1990|null|123611167|  60696724|    62914443|
|  0A|          人口集中地区| 平成|     2|   1990|null| 78152452|  38564229|    39588223|
|  0B|人口集中地区以外の地区| 平成|     2|   1990|null| 45458715|  22132495|    23326220|
|  01|                北海道| 平成|     2|   1990|null|  5643647|   2722988|     2920659|
|  02|                青森県| 平成|     2|   1990|null|  1482873|    704758|      778115|
|  03|                岩手県| 平成|     2|   1990|null|  1416928|    680197|      736731|
|  04|                宮城県| 平成|     2|   1990|null|  2248558|   1105103|     1143455|
|  05|                秋田県| 平成|     2|   1990|null|  1227478|    584678|      642800|
|  06|                山形県| 平成|     2|   1990|null|  1258390|

In [10]:
import pyspark.sql.functions as sf
df.where(df.gengo == "平成").groupBy("kenmei")\
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).show() 

+----------------------+--------------------+--------------------+
|                kenmei|            male_avg|          female_avg|
+----------------------+--------------------+--------------------+
|人口集中地区以外の地区|2.0976203166666668E7|2.2272045166666668E7|
|                佐賀県|            408192.5|            456442.5|
|                栃木県|   987741.8333333334|            999415.5|
|                京都府|  1268325.3333333333|  1360099.3333333333|
|                香川県|   485871.8333333333|   523763.6666666667|
|                愛媛県|   692188.3333333334|   774376.1666666666|
|                秋田県|   542928.3333333334|            604578.5|
|                広島県|           1387308.5|  1478006.8333333333|
|                宮崎県|            542386.5|            608793.0|
|              鹿児島県|            818506.0|            929134.0|
|                埼玉県|  3492880.3333333335|  3443447.8333333335|
|                三重県|            893167.5|   944959.6666666666|
|                島根県|            356034.

In [12]:
df_after_t = df.where(df.gengo == "平成").groupBy("kenmei")\
    .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("jinko_female_avg"))\
        .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")

In [14]:
df_after_t.show()

+--------+-----------------+-----------------+
|  kenmei|         male_avg| jinko_female_avg|
+--------+-----------------+-----------------+
|  鳥取県|287885.3333333333|314291.3333333333|
|  島根県|         356034.5|388621.6666666667|
|  高知県|372268.1666666667|         418517.0|
|  徳島県|383399.1666666667|         423152.0|
|  福井県|         395512.5|420182.6666666667|
|  佐賀県|         408192.5|         456442.5|
|  山梨県|425777.8333333333|441831.1666666667|
|  香川県|485871.8333333333|523763.6666666667|
|和歌山県|         490624.0|547112.3333333334|
|  富山県|         532857.0|573049.8333333334|
|  宮崎県|         542386.5|         608793.0|
|  秋田県|542928.3333333334|         604578.5|
|  石川県|         566064.0|         604518.5|
|  大分県|571530.6666666666|638773.6666666666|
|  山形県|         583603.5|627811.6666666666|
|  沖縄県|         654622.0|679050.6666666666|
|  岩手県|659592.6666666666|714973.1666666666|
|  滋賀県|         662391.0|         680326.0|
|  奈良県|671178.6666666666|734736.6666666666|
|  青森県|675238.6666666666